In [38]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# !pip install datasets
# !pip install evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import torch

In [39]:
ds = load_dataset("EdinburghNLP/xsum", trust_remote_code=True, streaming=False)

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [4]:
sample = ds['train'][0]

In [6]:
print(sample.keys())
print('')
print(sample['document'])
print('')
print(sample['summary'])

dict_keys(['document', 'summary', 'id'])

The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - an

In [10]:
summarizer = pipeline('summarization', model='facebook/bart-large-cnn')

In [11]:
summary = summarizer(sample['document'])

In [48]:
print(sample['summary'])
print('')
print(summary[0]['summary_text'])

del summarizer

Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank.

Many roads in Peeblesshire remain badly affected by standing water. The full cost of damage in Newton Stewart is still being assessed. First Minister Nicola Sturgeon visited the area to inspect the damage. A flood alert remains in place across the Borders because of the constant rain.


NameError: name 'summarizer' is not defined

In [25]:
# Manual pipeline

# tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
# model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')
# tokenized_text = tokenizer(sample['document'], return_tensors='pt')
# output = model.generate(**tokenized_text, num_beams=4, max_length=150, early_stopping=True)
# decoded = tokenizer.decode(output[0], skip_special_tokens=True)
# print(decoded)

# del tokenizer, model

C:\ProgramData\anaconda3\envs\env_dl_cv_transformer_sentpiece\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. First Minister Nicola Sturgeon visited the area to inspect the damage. Labour Party's deputy Scottish leader Alex Rowley was in Hawick on Monday to see the situation first hand. He said it was important to get the flood protection plan right.


In [51]:
sample2 = ds['train'][2]
print(sample2['document'])
print('')
print(sample2['summary'])

Ferrari appeared in a position to challenge until the final laps, when the Mercedes stretched their legs to go half a second clear of the red cars.
Sebastian Vettel will start third ahead of team-mate Kimi Raikkonen.
The world champion subsequently escaped punishment for reversing in the pit lane, which could have seen him stripped of pole.
But stewards only handed Hamilton a reprimand, after governing body the FIA said "no clear instruction was given on where he should park".
Belgian Stoffel Vandoorne out-qualified McLaren team-mate Jenson Button on his Formula 1 debut.
Vandoorne was 12th and Button 14th, complaining of a handling imbalance on his final lap but admitting the newcomer "did a good job and I didn't".
Mercedes were wary of Ferrari's pace before qualifying after Vettel and Raikkonen finished one-two in final practice, and their concerns appeared to be well founded as the red cars mixed it with the silver through most of qualifying.
After the first runs, Rosberg was ahead, 

In [24]:
bertscore = evaluate.load('bertscore')

In [55]:
# Compare summarizer to reference
# predictions = [summary[0]['summary_text']]
# references = [sample['summary']]

# Compare full text to summary
references = [sample['document']]
predictions = [sample['summary']]

# Sanity check
# references = [sample['summary']]
# predictions = [sample['summary']]

# Sanity check
# references = [sample['document']]
# predictions = [sample2['document']]

# Sanity check
# references = [sample['summary']]
# predictions = [sample2['summary']]

# references = ['I hate you']
# predictions = ['I love you']

results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")
results

{'precision': [0.8364526629447937],
 'recall': [0.6436858773231506],
 'f1': [0.7275167107582092],
 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.41.2)'}

In [ ]:
bleurt = evaluate.load("bleurt", module_type="metric")

In [60]:
# Compare summarizer to reference
# predictions = [summary[0]['summary_text']]
# references = [sample['summary']]

# Compare full text to summary
references = [sample['document']]
predictions = [sample['summary']]

# Sanity check
references = [sample['summary']]
predictions = [sample['summary']]

# Sanity check
# references = [sample['document']]
# predictions = [sample2['document']]

# Sanity check
# references = [sample['summary']]
# predictions = [sample2['summary']]

# references = ['I hate you']
# predictions = ['I love you']

results = bleurt.compute(predictions=predictions, references=references)
results

{'scores': [0.8976556062698364]}

In [61]:
del bertscore, bleurt

In [62]:
nli = pipeline("text-classification", model="FacebookAI/roberta-large-mnli")

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [92]:
reference = sample['summary']
prediction = summary[0]['summary_text']

# Compare full text to summary
reference = sample['document']
prediction = sample['summary']

# Sanity check
reference = sample['summary']
prediction = sample['summary']

# Sanity check
reference = sample['document']
prediction = sample2['document']

# Sanity check
reference = sample['summary']
prediction = sample2['summary']

# reference = 'I hate you'
# prediction = 'I love you'

# reference = 'I like you'
# prediction = 'I love you'

# reference = 'I love you'
# prediction = 'I want to be with you'

print(f"{reference} </s> {prediction}")
results = nli(f"{reference} </s> {prediction}", truncation=True)
results

Clean-up operations are continuing across the Scottish Borders and Dumfries and Galloway after flooding caused by Storm Frank. </s> Lewis Hamilton stormed to pole position at the Bahrain Grand Prix ahead of Mercedes team-mate Nico Rosberg.


[{'label': 'CONTRADICTION', 'score': 0.5319812893867493}]

In [ ]:
del nli

In [15]:
bleu = evaluate.load("bleu")

predictions = [summary[0]['summary_text']]
# predictions = [sample['summary']]
references = [sample['summary']]
results = bleu.compute(predictions=predictions, references=references)
results

{'bleu': 0.0,
 'precisions': [0.09803921568627451, 0.02, 0.0, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 2.6842105263157894,
 'translation_length': 51,
 'reference_length': 19}

In [16]:
rouge = evaluate.load('rouge')

predictions = [summary[0]['summary_text']]
# predictions = [sample['summary']]
references = [sample['summary']]
results = rouge.compute(predictions=predictions, references=references)
results

{'rouge1': 0.12121212121212122,
 'rouge2': 0.03125,
 'rougeL': 0.0909090909090909,
 'rougeLsum': 0.0909090909090909}

In [ ]:
evaluate.list_evaluation_modules()